#### Instructions to run -  Use llm-zoomcamp-env from Conda

In [34]:
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333/") # connecting to local qdrant instance

In [2]:
# !pip install pandas
import pandas as pd
documents = pd.read_csv("./home_office_expense_deduction_scenario.csv")

In [3]:
documents

,scenario,can_claim
0,"Alex uses their home internet 30% for work, co...",Yes
1,Ben exclusively uses their home phone for pers...,No
2,"Chloe purchases a new router for $250, used 70...",Yes
3,David's employer directly pays for his home in...,No
4,Emily is seeking new employment and uses her h...,No
...,...,...
193,"Leo's home phone bill is minimal, and he wants...",No
194,Michelle works from home and uses the fixed ra...,No
195,"Nathan's home internet is consistently slow, a...",No
196,Olivia's home internet is shared with several ...,No


In [4]:
from fastembed import TextEmbedding
import json

EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

In [5]:
model_handle="jinaai/jina-embeddings-v2-small-en"

In [ ]:
collection_name = "ato-rag"

qd_client.create_collection(
    collection_name=collection_name, 
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)


UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `ato-rag` already exists!"},"time":0.001092022}'

In [23]:
points=[]
for row in documents.itertuples(index=True, name='Row'):
    # print(f"can_claim: {row.can_claim}, Scenario: {row.scenario}")

    point=models.PointStruct(
            id=row.Index,
            vector=models.Document(text=row.scenario, model=model_handle),
            payload={
                "scenario": row.scenario,
                "can_claim": row.can_claim
            }
        )
    points.append(point)



In [47]:
points[0]

PointStruct(id=0, vector=Document(text="Alex uses their home internet 30% for work, connecting to their office's virtual desktop. They have all bills and a 4-week diary of work use. They are using the actual cost method.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'scenario': "Alex uses their home internet 30% for work, connecting to their office's virtual desktop. They have all bills and a 4-week diary of work use. They are using the actual cost method.", 'can_claim': 'Yes'})

In [ ]:

qd_client.upsert(collection_name=collection_name, 
              points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="can_claim", # Don't know if it will work
    field_schema="keyword"
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
def qd_search(query, limit=1):

    results = qd_client.query_points(
        collection_name=collection_name,
        query = models.Document(
            text = query,
            model = model_handle
        ),
        limit = limit,
        with_payload = True # to get metadata in the results
    )

    return results

### Making it generic search function

In [ ]:
# !pip install openai
# !pip install python-dotenv

In [32]:
import os
from dotenv import load_dotenv
load_dotenv() ## THIS WILL LOAD THE .env FILE with OPENAI_API_KEY

from openai import OpenAI
openai_client = OpenAI()

In [56]:
def qd_search(query, limit=1):
    print('vector search using qdrant')
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query = models.Document(
            text = query,
            model = model_handle
        ),
        limit = limit,
        with_payload = True # to get metadata in the results
    )

    # Maybe add filter when more number of sections

    results=[]

    for point in query_points.points:
        results.append(point.payload)

    return results

In [33]:
def build_prompt(query, search_results):
    prompt_template = """
You're a ATO tax filing assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"scenario: {doc['scenario']}\can_claim: {doc['can_claim']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [69]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [ ]:
def rag(query):
    results = []
    search_results = qd_search(query, limit=3)
    # print(search_results)
    
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer


In [68]:
openai_client.models.list()

SyncPage[Model](data=[Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai')], object='list')

In [70]:
rag('I work from home 3 days a week. I have a separate home office. Can I claim the home office expenses?')

vector search using qdrant
[{'scenario': 'Michelle works from home and uses the fixed rate method. She wants to claim a separate deduction for her home phone calls.', 'can_claim': 'No'}, {'scenario': "Jack's employer provides him with a monthly allowance for home office expenses, which is meant to cover internet and phone. He wants to claim separate deductions.", 'can_claim': 'No'}, {'scenario': 'Grace claims the fixed rate method for working from home expenses, and also wants to claim a separate deduction for her home internet usage.', 'can_claim': 'No'}]


'No, you cannot claim separate deductions for home office expenses if you use the fixed rate method for working from home.'